In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_dir='../input/fer2013/train'
test_dir='../input/fer2013/test'

print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))

# Visualisation of the IMAGES

In [ ]:
#ANGRY

img=cv2.imread('../input/fer2013/train/angry/Training_10290703.jpg')
plt.imshow(img)
img.shape

In [ ]:
#DISGUST
img=cv2.imread('../input/fer2013/train/disgust/Training_11550217.jpg')
plt.imshow(img)

In [ ]:
#FEAR

img=cv2.imread('../input/fer2013/train/fear/Training_10127393.jpg')
plt.imshow(img)

In [ ]:
#HAPPY
img=cv2.imread('../input/fer2013/train/happy/Training_10229138.jpg')
plt.imshow(img)

In [ ]:
#NEUTRAL
img=cv2.imread('../input/fer2013/train/neutral/Training_10169962.jpg')
plt.imshow(img)

In [ ]:
#SAD
img=cv2.imread('../input/fer2013/train/sad/Training_10134025.jpg')
plt.imshow(img)

In [ ]:
#SURPRISE
img=cv2.imread('../input/fer2013/train/surprise/Training_10218787.jpg')
plt.imshow(img)

# DATA AUGMENTATION

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1.0/255.0, 
                           rotation_range=40,
                           width_shift_range=0.2,
                           height_shift_range=0.2,
                           horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1.0/255.0)

trainDatagen=train_datagen.flow_from_directory(train_dir,
                                 target_size=(48,48),
                                 batch_size=100,
                                 class_mode='categorical',
                                 color_mode='grayscale')
                                
valDatagen=val_datagen.flow_from_directory(test_dir,
                                      target_size=(48,48),
                                      batch_size=100,
                                      class_mode='categorical',
                                      color_mode='grayscale')

model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(48,48,1),padding='same'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.Dropout(0.3),
                                 tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.Dropout(0.3),
                                 tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.BatchNormalization(),
                                 
                                 tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.Dropout(0.3),
                                 tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.Dropout(0.5),
                                  
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(512,activation='relu'),

                                 tf.keras.layers.Dense(7,activation='softmax')])
model.summary()

In [ ]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Convolution2D(filters=64,kernel_size=3 ,activation='relu', input_shape=[48, 48, 1]))
cnn.add(tf.keras.layers.Convolution2D(filters=64,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Convolution2D(filters=128,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Convolution2D(filters=128,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Convolution2D(filters=256,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Convolution2D(filters=256,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Convolution2D(filters=512,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Convolution2D(filters=512,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=512, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))


cnn.add(tf.keras.layers.Dense(units=7, activation='softmax'))

In [ ]:
class mycallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('accuracy')>=0.99):
            self.model.stop_training=True
            print("MODEL AS REACHED AMAZING ACCURACY")

In [ ]:
callbacks=mycallbacks()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
history=model.fit_generator(trainDatagen,epochs=100,validation_data=valDatagen,callbacks=[callbacks],shuffle=True)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()